In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.models.segmentation import deeplabv3_resnet50
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from google.colab import drive

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Define dataset paths
BASE_DIR = "/content/drive/MyDrive/Clothing Segmentation/dataset"
TRAIN_IMAGES_DIR = os.path.join(BASE_DIR, "train", "images")
TRAIN_MASKS_DIR = os.path.join(BASE_DIR, "train", "masks")
VAL_IMAGES_DIR = os.path.join(BASE_DIR, "val", "images")
VAL_MASKS_DIR = os.path.join(BASE_DIR, "val", "masks")

In [4]:
# Image size and classes
IMG_HEIGHT, IMG_WIDTH = 256, 256
NUM_CLASSES = 59
BATCH_SIZE = 8

In [5]:
# Transformations
transform = transforms.Compose([
    transforms.Resize((IMG_HEIGHT, IMG_WIDTH)),
    transforms.ToTensor(),
])

In [6]:
class SegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.image_filenames = sorted(os.listdir(image_dir))
        self.mask_filenames = sorted(os.listdir(mask_dir))
        self.transform = transform

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, idx):
        img_path = os.path.join(self.image_dir, self.image_filenames[idx])
        mask_path = os.path.join(self.mask_dir, self.mask_filenames[idx])

        image = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path).convert("L")

        if self.transform:
            image = self.transform(image)
            mask = self.transform(mask) * (NUM_CLASSES - 1)  # Normalize mask

        return image, mask.squeeze().long()

In [7]:
# Data loaders
train_dataset = SegmentationDataset(TRAIN_IMAGES_DIR, TRAIN_MASKS_DIR, transform)
val_dataset = SegmentationDataset(VAL_IMAGES_DIR, VAL_MASKS_DIR, transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)


In [8]:
# Load DeepLabV3+ model
model = deeplabv3_resnet50(pretrained=True)
model.classifier[4] = nn.Conv2d(256, NUM_CLASSES, kernel_size=1)  # Adjust output classes
model = model.to('cuda' if torch.cuda.is_available() else 'cpu')

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=DeepLabV3_ResNet50_Weights.COCO_WITH_VOC_LABELS_V1`. You can also use `weights=DeepLabV3_ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/deeplabv3_resnet50_coco-cd0a2569.pth" to /root/.cache/torch/hub/checkpoints/deeplabv3_resnet50_coco-cd0a2569.pth
100%|██████████| 161M/161M [00:00<00:00, 181MB/s]


In [9]:
# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [10]:
# Define the save path in Google Drive
CHECKPOINT_DIR = "/content/drive/MyDrive/Clothing Segmentation/checkpoints"
os.makedirs(CHECKPOINT_DIR, exist_ok=True)  # Ensure the directory exists

In [11]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20):
    device = 'cuda' if torch.cuda.is_available() else 'cpu'
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for images, masks in train_loader:
            images, masks = images.to(device), masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)['out']
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/len(train_loader):.4f}")

    # Save the model inside 'checkpoints' directory
    torch.save(model.state_dict(), os.path.join(CHECKPOINT_DIR, "deeplabv3_pytorch.pth"))
    print(f"Model saved at: {CHECKPOINT_DIR}/deeplabv3_pytorch.pth")


In [12]:
# Train the model
train_model(model, train_loader, val_loader, criterion, optimizer, epochs=20)

Epoch 1/20, Loss: 2.5077
Epoch 2/20, Loss: 1.4860
Epoch 3/20, Loss: 0.9611
Epoch 4/20, Loss: 0.6726
Epoch 5/20, Loss: 0.5173
Epoch 6/20, Loss: 0.4199
Epoch 7/20, Loss: 0.3571
Epoch 8/20, Loss: 0.3092
Epoch 9/20, Loss: 0.2733
Epoch 10/20, Loss: 0.2487
Epoch 11/20, Loss: 0.2248
Epoch 12/20, Loss: 0.2076
Epoch 13/20, Loss: 0.1976
Epoch 14/20, Loss: 0.1836
Epoch 15/20, Loss: 0.1858
Epoch 16/20, Loss: 0.1768
Epoch 17/20, Loss: 0.1758
Epoch 18/20, Loss: 0.1571
Epoch 19/20, Loss: 0.1440
Epoch 20/20, Loss: 0.1352
Model saved at: /content/drive/MyDrive/Clothing Segmentation/checkpoints/deeplabv3_pytorch.pth


In [14]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [15]:
# Load the pretrained model
model = deeplabv3_resnet50(pretrained=True)

# Modify the classifier to match NUM_CLASSES (59)
model.classifier[4] = torch.nn.Conv2d(256, 59, kernel_size=(1, 1), stride=(1, 1))

# If using aux_classifier, modify it as well
if model.aux_classifier:
    model.aux_classifier[4] = torch.nn.Conv2d(256, 59, kernel_size=(1, 1), stride=(1, 1))

# Move to device
model = model.to(device)


In [17]:
CHECKPOINT_PATH = "/content/drive/MyDrive/Clothing Segmentation/checkpoints/deeplabv3_pytorch.pth"
EPOCHS = 30  # Further training for 30 epochs

In [20]:
# If a checkpoint exists, load the saved weights.
if os.path.exists(CHECKPOINT_PATH):
    checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)

    # Remove aux_classifier keys from checkpoint
    filtered_checkpoint = {k: v for k, v in checkpoint.items() if "aux_classifier" not in k}

    # Load the remaining state dict with strict=False
    model.load_state_dict(filtered_checkpoint, strict=False)

    print("Checkpoint loaded successfully, ignoring aux_classifier mismatches.")

    print(f"Loaded checkpoint from: {CHECKPOINT_PATH}")
else:
    print("No checkpoint found, starting training from scratch.")

<ipython-input-20-168f6db4c439>:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)


Checkpoint loaded successfully, ignoring aux_classifier mismatches.
Loaded checkpoint from: /content/drive/MyDrive/Clothing Segmentation/checkpoints/deeplabv3_pytorch.pth


In [21]:
# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [22]:
# Training function
def train_model(model, train_loader, val_loader, criterion, optimizer, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0.0
        for images, masks in train_loader:
            images = images.to(device)
            masks = masks.to(device)
            optimizer.zero_grad()
            outputs = model(images)['out']
            loss = criterion(outputs, masks)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        avg_loss = total_loss / len(train_loader)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
    return model


In [23]:
# Train the model for further 30 epochs
model = train_model(model, train_loader, val_loader, criterion, optimizer, EPOCHS)

# Save the updated model checkpoint to Google Drive
torch.save(model.state_dict(), CHECKPOINT_PATH)
print(f"Updated checkpoint saved at: {CHECKPOINT_PATH}")

Epoch 1/30, Loss: 0.1328
Epoch 2/30, Loss: 0.1497
Epoch 3/30, Loss: 0.1383
Epoch 4/30, Loss: 0.1304
Epoch 5/30, Loss: 0.1205
Epoch 6/30, Loss: 0.1254
Epoch 7/30, Loss: 0.1126
Epoch 8/30, Loss: 0.1056
Epoch 9/30, Loss: 0.1016
Epoch 10/30, Loss: 0.0991
Epoch 11/30, Loss: 0.0968
Epoch 12/30, Loss: 0.0950
Epoch 13/30, Loss: 0.0932
Epoch 14/30, Loss: 0.0914
Epoch 15/30, Loss: 0.0904
Epoch 16/30, Loss: 0.0891
Epoch 17/30, Loss: 0.0875
Epoch 18/30, Loss: 0.0863
Epoch 19/30, Loss: 0.0852
Epoch 20/30, Loss: 0.0845
Epoch 21/30, Loss: 0.0836
Epoch 22/30, Loss: 0.0826
Epoch 23/30, Loss: 0.0815
Epoch 24/30, Loss: 0.0806
Epoch 25/30, Loss: 0.0799
Epoch 26/30, Loss: 0.0795
Epoch 27/30, Loss: 0.0787
Epoch 28/30, Loss: 0.0780
Epoch 29/30, Loss: 0.0809
Epoch 30/30, Loss: 0.0843
Updated checkpoint saved at: /content/drive/MyDrive/Clothing Segmentation/checkpoints/deeplabv3_pytorch.pth
